In [1]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
color_list = ['salmon', 'limegreen', 'mediumturquoise', 'cornflowerblue', 'fuchsia', 'khaki']

import copy as cp
import pickle
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display


import RandomPEPS as rpeps
import StructureMatrixGenerator as smg
import trivialSimpleUpdate as tsu
import DoubleEdgeFactorGraphs as defg
import SimpleUpdate as su
import bmpslib as bmps

## Bethe free energy PEPS calculations

In this code we will initialize a random quantum state with a random $N\times M$ PEPS and ivolve it with imaginary time (ITE) according to the **Antiferromagnetic Heisenberd Hamiltonian**

$
\begin{align}
H = J\sum_{\langle i,j \rangle} \mathbf{S}_i\cdot \mathbf{S}_j
\end{align}
$

with $J = -1$. Then, after the PEPS converges to a ground-state representation transform the PEPS tensor network into its dual **Double-Edge Factor Graph (DEFG)** and run BP until it reaches a fixed point. Using the fixed point messages we calculate the factor and node beliefs and then using calculate the Bethe free energy following the next formula:

$
\begin{align}
F_{Bethe}[q] &=\sum_{\alpha}\langle \log \left(\frac{q_{\alpha}(\mathbf{x}_{\alpha})}{f_{\alpha}(\mathbf{x}_{\alpha})} \right) \rangle_q +  \sum_{i} (1-d_i)\langle  \log(q_i(x_i))\rangle_q
\end{align}
$

where $q_{\alpha}(\mathbf{x}_{\alpha}), q_i(x_i)$ are the factor and node beliefs (approximated marginal) respectively and $d_i$ indicates the number of factor naighbors of the $i^{th}$ node.

In [ ]:
# SU and BP parameters
N, M = 4, 4                                                 # NxM PEPS
bc = 'open'                                                   # boundary conditions
dw = 1e-6                                                     # maximal error allowed between two-body RDMS
d = 2                                                         # tensor network physical bond dimension
bond_dimension = 2                                   # maximal virtual bond dimensions allowed for truncation
t_max = 100                                                   # maximal number of BP iterations
epsilon = 1e-6                                              # convergence criteria for BP messages (not used)
dumping = 0.2                                                  # BP messages dumping between [0, 1]
iterations = 100                                              # maximal number of tSU iterations
BPU_iterations = 100                                          # maximal number of BPU iterations
num_experiments = 1                                          # number of random experiments for each bond dimension
smat, _ = smg.finitePEPSobcStructureMatrixGenerator(N, M)     # generating the PEPS structure matrix
tensors, weights = smg.randomTensornetGenerator(smat, d, D_max)
n, m = smat.shape

In [ ]:
# ITE parameters
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
X = np.array([[0, 1], [1, 0]])
Sz = 0.5 * Z
Sy = 0.5 * Y
Sx = 0.5 * X
Opi = [Sx, Sy, Sz]
Opj = [Sx, Sy, Sz]
Op_field = np.eye(d)
interactionConstants = [-1] * m
timeStep = [0.1, 0.05, 0.01, 0.005, 0.001]

In [ ]:
# run the Simple Update algorithm
for dt in timeStep:
    for i in range(iterations):
        weights_prev = cp.deepcopy(weights)
        tensors_next, weights_next = su.simpleUpdate(tensors,
                                                     weights,
                                                     dt,
                                                     interactionConstants,
                                                     0,
                                                     Opi,
                                                     Opj,
                                                     Op_field,
                                                     smat,
                                                     Bond_dimension,
                                                     'SU')
        if np.sum(np.abs(np.asarray(weights_prev) - np.asarray(weights_next))) < dt * 1e-2:
            tensors = tensors_next
            weights = weights_next
            break
        tensors = tensors_next
        weights = weights_next

In [ ]:
# run few iterations of trivial Simple Update algorithm in order to get the "quasi-canonical" PEPS representation
# of the AFH ground-state
errors = []
for i in range(iterations):
    weights_prev = cp.deepcopy(weights)
    tensors_next, weights_next = tsu.trivialsimpleUpdate(tensors,
                                                         weights,
                                                         smat,
                                                         D_max)
    error = np.sum(np.abs(np.asarray(weights) - np.asarray(weights_next)))
    errors.append(error)
    if error < dw:
        print('The final error is: {} in {} iterations'.format(error, i))
        tensors = tensors_next
        weights = weights_next
        break
    tensors = tensors_next
    weights = weights_next

In [ ]:
# save the fixed-point Tensor Net
tensors_fixedPoint = cp.deepcopy(tensors)
weights_fixedPoint = cp.deepcopy(weights)

In [ ]:
# constructing the dual double-edge factor graph and run BP until it converge
graph = defg.defg()
graph = su.TNtoDEFGtransform(graph, tensors, weights, smat)
graph.sumProduct(t_max, epsilon, dumping, initializeMessages=1, printTime=1, RDMconvergence=0)

In [ ]:
# calculate the DEFG node and factor beliefs
graph.calculateFactorsBeliefs()
graph.calculateNodesBeliefs()

In [ ]:
# calculate the Bethe free energy
def Bethe_Free_Energy(defg):
    factors = defg.factors
    nodes = defg.nodes
    
    factorBeliefs = defg.factorsBeliefs
    nodeBeliefs = defg.nodesBeliefs
    
    Bethe_energy_term = 0
    Bethe_entropy_term = 0
    
    for f in factors.keys():
        factor = factors[f][1]
        fbelief = factorBeliefs[f]
        Bethe_energy_term += np.sum(fbelief * np.log10(fbelief / factor))
    for n in nodes.keys():
        d_n = len(nodes[n][1])
        nbelief = nodeBeliefs(n)
        Bethe_entropy_term += (1 - d_n) * np.sum(nbelief * np.log10(nbelief))
        
    return Bethe_energy_term + Bethe_entropy_term

In [ ]:
bethe = Bethe_Free_Energy(graph)
print(bethe)